In [2]:
### 1. Weather model

### First, load the dataset from the weatherinszeged table from Thinkful's database.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import statsmodels.api as sm

import warnings
warnings.filterwarnings('ignore')

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'weatherinszeged'
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
weather_df = pd.read_sql_query('select * from weatherinszeged',con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()

In [3]:
## As in the previous lesson, build a linear regression model where your target variable is the difference between the 
## apparenttemperature and the temperature. As explanatory variables, use humidity and windspeed. Now, estimate your model using 
## OLS. What are the R-squared and adjusted R-squared values? Do you think they are satisfactory? Why?

Y = weather_df['apparenttemperature'] - weather_df['temperature']
X = weather_df[['humidity','windspeed']]
X = sm.add_constant(X)
results = sm.OLS(Y, X).fit()
results.summary()

# r-squared values are 28.8% This is pretty low indicating the model is not doing much to explain the data.

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.288
Model:                            OLS   Adj. R-squared:                  0.288
Method:                 Least Squares   F-statistic:                 1.949e+04
Date:                Mon, 31 Mar 2025   Prob (F-statistic):               0.00
Time:                        16:28:08   Log-Likelihood:            -1.7046e+05
No. Observations:               96453   AIC:                         3.409e+05
Df Residuals:                   96450   BIC:                         3.409e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.4381      0.021    115.948      0.000       2.397       2.479
humidity      -3.0292      0.024   -126.479      0.000      -3.076      -2.982
windspeed     -0.1193      0.001   -176.164      0.000      -0.121      -0.118
==============================================================================
Omnibus:                     3935.747   Durbin-Watson:                   0.267
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4613.311
Skew:                          -0.478   Prob(JB):                         0.00
Kurtosis:                       3.484   Cond. No.                         88.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [4]:
### Next, include the interaction of humidity and windspeed to the model above and estimate the model using OLS. Now, what is 
### the R-squared of this model? Does this model improve upon the previous one?

weather_df['humidity_windspeed_interaction'] = weather_df.humidity * weather_df.windspeed
Y = weather_df['apparenttemperature'] - weather_df['temperature']
X = weather_df[['humidity','windspeed', 'humidity_windspeed_interaction']]
X = sm.add_constant(X)
results = sm.OLS(Y, X).fit()
results.summary()

# Yes this improved the model slightly moving the r values up to 34%.

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.341
Model:                            OLS   Adj. R-squared:                  0.341
Method:                 Least Squares   F-statistic:                 1.666e+04
Date:                Mon, 31 Mar 2025   Prob (F-statistic):               0.00
Time:                        16:30:43   Log-Likelihood:            -1.6669e+05
No. Observations:               96453   AIC:                         3.334e+05
Df Residuals:                   96449   BIC:                         3.334e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
const                              0.0839      0.033      2.511      0.012       0.018       0.149
humidity                           0.1775      0.043      4.133      0.000       0.093       0.262
windspeed                          0.0905      0.002     36.797      0.000       0.086       0.095
humidity_windspeed_interaction    -0.2971      0.003    -88.470      0.000      -0.304      -0.291
==============================================================================
Omnibus:                     4849.937   Durbin-Watson:                   0.265
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9295.404
Skew:                          -0.378   Prob(JB):                         0.00
Kurtosis:                       4.320   Cond. No.                         193.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [5]:
### Add visibility as an additional explanatory variable to the first model and estimate it. Did R-squared increase? What about 
### adjusted R-squared? Compare the differences put on the table by the interaction term and the visibility in terms of the 
### improvement in the adjusted R-squared. Which one is more useful?

Y = weather_df['apparenttemperature'] - weather_df['temperature']
X = weather_df[['humidity','windspeed', 'visibility']]
X = sm.add_constant(X)
results = sm.OLS(Y, X).fit()
results.summary()

# This method at 30% is about 4% lower than using the interaction term. So as it stands this is less useful.

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.304
Model:                            OLS   Adj. R-squared:                  0.303
Method:                 Least Squares   F-statistic:                 1.401e+04
Date:                Mon, 31 Mar 2025   Prob (F-statistic):               0.00
Time:                        16:32:20   Log-Likelihood:            -1.6938e+05
No. Observations:               96453   AIC:                         3.388e+05
Df Residuals:                   96449   BIC:                         3.388e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.5756      0.028     56.605      0.000       1.521       1.630
humidity      -2.6066      0.025   -102.784      0.000      -2.656      -2.557
windspeed     -0.1199      0.001   -179.014      0.000      -0.121      -0.119
visibility     0.0540      0.001     46.614      0.000       0.052       0.056
==============================================================================
Omnibus:                     3833.895   Durbin-Watson:                   0.282
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4584.022
Skew:                          -0.459   Prob(JB):                         0.00
Kurtosis:                       3.545   Cond. No.                         131.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [6]:
### Choose the best one from the three models above with respect to their AIC and BIC scores. Validate your choice by discussing
### your justification with your mentor.

# 'Model'         AIC         BIC
# 'default'       3.409e+05   3.409e+05              
# 'interaction'   3.334e+05   3.334e+05         
# 'visibility'    3.388e+05   3.388e+05       

# Factoring in these scores I would say using the interaction between humidity and windspeed produces the best model. It has the
# highest score and now looking at aic and bic we can see it gives the lowest value of the 3 models. Which for these scores is 
# the prefered outcome.

In [7]:
### 2. House prices model
### Load the houseprices data from Thinkful's database.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import statsmodels.api as sm

import warnings
warnings.filterwarnings('ignore')

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
house_df = pd.read_sql_query('select * from houseprices',con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()

In [8]:
### Run your house prices model again and assess the goodness of fit of your model using an F-test, R-squared, adjusted 
### R-squared, AIC, and BIC. Do you think that your model is satisfactory? If so, why?

house_df = pd.concat([house_df,pd.get_dummies(house_df.centralair, prefix = "centralair", drop_first = True)], axis = 1)
dummy_column_names = list(pd.get_dummies(house_df.centralair, prefix = "centralair", drop_first = True).columns)
Y = house_df['saleprice']
X = house_df[['overallqual', 'grlivarea', 'totalbsmtsf', 'firstflrsf'] + dummy_column_names]
X = sm.add_constant(X)
results = sm.OLS(Y, X).fit()
results.summary()

# F-stat is 855, AIC 35,100, and BIC of 35,130. R-squared and Adjusted are ~75% so not low but would probably prefer a model
# that outputs a higher score.

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.746
Model:                            OLS   Adj. R-squared:                  0.745
Method:                 Least Squares   F-statistic:                     855.3
Date:                Mon, 31 Mar 2025   Prob (F-statistic):               0.00
Time:                        16:39:27   Log-Likelihood:                -17543.
No. Observations:                1460   AIC:                         3.510e+04
Df Residuals:                    1454   BIC:                         3.513e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -1.197e+05   5803.182    -20.632      0.000   -1.31e+05   -1.08e+05
overallqual   2.762e+04   1048.864     26.335      0.000    2.56e+04    2.97e+04
grlivarea       46.9016      2.760     16.990      0.000      41.487      52.317
totalbsmtsf     24.5082      4.447      5.511      0.000      15.785      33.232
firstflrsf      17.0189      5.160      3.299      0.001       6.898      27.140
centralair_Y  1.647e+04   4458.211      3.693      0.000    7720.514    2.52e+04
==============================================================================
Omnibus:                      491.366   Durbin-Watson:                   1.984
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            43573.793
Skew:                          -0.594   Prob(JB):                         0.00
Kurtosis:                      29.737   Cond. No.                     1.40e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.4e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [12]:
### In order to improve the goodness of fit of your model, try different model specifications by adding or removing some 
### variables.

house_df['totalsf'] = house_df['totalbsmtsf'] + house_df['firstflrsf'] + house_df['secondflrsf']
Y = np.log1p(house_df['saleprice'])
X = house_df[['overallqual', 'grlivarea', 'garagecars', 'totalsf'] + dummy_column_names]
X = sm.add_constant(X)
results = sm.OLS(Y, X).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.811
Model:                            OLS   Adj. R-squared:                  0.811
Method:                 Least Squares   F-statistic:                     1251.
Date:                Mon, 31 Mar 2025   Prob (F-statistic):               0.00
Time:                        16:49:05   Log-Likelihood:                 486.36
No. Observations:                1460   AIC:                            -960.7
Df Residuals:                    1454   BIC:                            -929.0
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           10.4374      0.024    429.135      0.000      10.390      10.485
overallqual      0.1181      0.005     24.278      0.000       0.109       0.128
grlivarea     8.361e-05    1.8e-05      4.645      0.000    4.83e-05       0.000
garagecars       0.1095      0.008     13.945      0.000       0.094       0.125
totalsf          0.0001   1.25e-05     10.652      0.000       0.000       0.000
centralair_Y     0.2176      0.019     11.210      0.000       0.180       0.256
==============================================================================
Omnibus:                      858.130   Durbin-Watson:                   2.025
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            24742.593
Skew:                          -2.207   Prob(JB):                         0.00
Kurtosis:                      22.679   Cond. No.                     1.89e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.89e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [13]:
### For each model that you try, get the goodness-of-fit metrics and compare your models with each other. Which model is the 
### best and why?

# Second model is an overall imporvement. R-squared values are up to 81% and F-statistic increased while both AIC and BIC
# decreased.